In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
import datetime
import time

In [2]:
house_df = pd.read_csv('../data/properties_2016_backup.csv')

D:\Program Files\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
# delete features with too many missing values
nan = house_df.isnull().sum()
nan = nan[nan < 2900000]
nan_feature = nan.index.tolist()
house_nan_df = house_df.loc[:,nan_feature]

In [13]:
# delete categorical features
cate_feature = ['airconditioningtypeid','fips','heatingorsystemtypeid','pooltypeid7','propertycountylandusecode','propertylandusetypeid',
                'propertyzoningdesc','rawcensustractandblock','regionidcity','regionidcounty','regionidneighborhood','regionidzip',
                'censustractandblock']

for i in cate_feature:
    nan_feature.remove(i)

nan_cate_feature = nan_feature

house_nan_cate_df = house_nan_df.loc[:,nan_cate_feature]

In [14]:
# delete highly correlated features

highcorr_feature = ['structuretaxvaluedollarcnt','threequarterbathnbr','finishedfloor1squarefeet','finishedsquarefeet12','finishedsquarefeet15', 'finishedsquarefeet50','taxvaluedollarcnt']

for i in highcorr_feature:
    nan_cate_feature.remove(i)

nan_cate_co_feature = nan_cate_feature

house_nan_co_cate_df = house_nan_cate_df.loc[:,nan_cate_co_feature]

# rename the dataframe, after deleting many features
house_new_df = house_nan_co_cate_df

In [15]:
house_new_df.shape

(2985217, 21)

In [16]:
nan_zero = 'poolcnt'
house_new_df[nan_zero] = house_new_df[nan_zero].fillna(value=0)

In [17]:
nan_mean = ['parcelid', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid','calculatedbathnbr', 'calculatedfinishedsquarefeet', 
            'fireplacecnt', 'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'latitude', 'longitude','lotsizesquarefeet',
            'roomcnt', 'unitcnt', 'yearbuilt', 'numberofstories','assessmentyear','landtaxvaluedollarcnt', 'taxamount']

house_new_df[nan_mean] = house_new_df[nan_mean].fillna(house_new_df[nan_mean].mean())

In [9]:
house_new_df.describe()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,...,longitude,lotsizesquarefeet,poolcnt,roomcnt,unitcnt,yearbuilt,numberofstories,assessmentyear,landtaxvaluedollarcnt,taxamount
count,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,...,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06
mean,1.332586e+07,2.209143e+00,3.088949e+00,5.784787e+00,2.299263e+00,1.827162e+03,1.168710e+00,2.244165e+00,1.823517e+00,3.837694e+02,...,-1.182019e+08,2.282281e+04,1.733656e-01,1.475018e+00,1.181171e+00,1.964262e+03,1.401464e+00,2.014999e+03,2.524780e+05,5.377607e+03
std,7.909966e+06,1.075683e+00,1.273410e+00,1.454807e+00,9.788902e-01,1.802765e+03,1.492758e-01,9.695674e-01,3.318279e-01,1.335086e+02,...,3.446550e+05,3.216019e+05,3.785631e-01,2.834938e+00,1.992334e+00,2.320484e+01,2.576771e-01,3.676098e-02,4.399356e+05,9.134915e+03
min,1.071172e+07,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,...,-1.194758e+08,1.000000e+02,0.000000e+00,0.000000e+00,1.000000e+00,1.801000e+03,1.000000e+00,2.000000e+03,1.000000e+00,1.340000e+00
25%,1.164371e+07,2.000000e+00,2.000000e+00,4.000000e+00,2.000000e+00,1.220000e+03,1.168710e+00,2.000000e+00,1.823517e+00,3.837694e+02,...,-1.183920e+08,5.874000e+03,0.000000e+00,0.000000e+00,1.000000e+00,1.950000e+03,1.401464e+00,2.015000e+03,7.710000e+04,2.478050e+03
50%,1.254509e+07,2.000000e+00,3.000000e+00,5.784787e+00,2.000000e+00,1.588000e+03,1.168710e+00,2.000000e+00,1.823517e+00,3.837694e+02,...,-1.181744e+08,7.296000e+03,0.000000e+00,0.000000e+00,1.000000e+00,1.964000e+03,1.401464e+00,2.015000e+03,1.718430e+05,4.027760e+03
75%,1.409712e+07,3.000000e+00,4.000000e+00,7.000000e+00,3.000000e+00,2.120000e+03,1.168710e+00,3.000000e+00,1.823517e+00,3.837694e+02,...,-1.179502e+08,1.350200e+04,0.000000e+00,0.000000e+00,1.181171e+00,1.981000e+03,1.401464e+00,2.015000e+03,3.022000e+05,6.166200e+03
max,1.696019e+08,2.000000e+01,2.000000e+01,1.200000e+01,2.000000e+01,9.525760e+05,9.000000e+00,2.000000e+01,2.500000e+01,7.749000e+03,...,-1.175543e+08,3.282638e+08,1.000000e+00,9.600000e+01,9.970000e+02,2.015000e+03,4.100000e+01,2.016000e+03,9.024622e+07,3.458861e+06


In [18]:
cols_to_norm = ['bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr','calculatedfinishedsquarefeet', 'fireplacecnt',
                'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'latitude', 'longitude','lotsizesquarefeet', 'poolcnt', 'roomcnt', 
                'unitcnt', 'yearbuilt', 'numberofstories','assessmentyear', 'landtaxvaluedollarcnt', 'taxamount']


for col in cols_to_norm:
    house_new_df[col] = (house_new_df[col] - house_new_df[col].mean())/(house_new_df[col].std())

# house_new_df[cols_to_norm] = house_new_df[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.std())

In [19]:
house_new_df.describe()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,...,longitude,lotsizesquarefeet,poolcnt,roomcnt,unitcnt,yearbuilt,numberofstories,assessmentyear,landtaxvaluedollarcnt,taxamount
count,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,...,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06
mean,1.332586e+07,1.128195e-12,1.522309e-12,-2.351569e-12,-8.170124e-13,1.252365e-13,-3.143632e-10,4.977838e-12,-1.806998e-10,-1.037957e-10,...,3.120837e-10,1.904334e-13,-6.835842e-14,2.637716e-14,-7.390089e-12,-9.917057e-11,1.025097e-10,-3.431327e-08,7.377439e-13,-2.490102e-13
std,7.909966e+06,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,1.071172e+07,-2.053712e+00,-2.425731e+00,-3.288950e+00,-1.327281e+00,-1.012979e+00,-1.130190e+00,-1.283217e+00,-5.495368e+00,-2.874491e+00,...,-3.696003e+00,-7.065507e-02,-4.579570e-01,-5.203000e-01,-9.093410e-02,-7.035673e+00,-1.558011e+00,-4.080267e+02,-5.738954e-01,-5.885405e-01
25%,1.164371e+07,-1.944279e-01,-8.551442e-01,-1.226820e+00,-3.057161e-01,-3.367950e-01,-3.143712e-10,-2.518289e-01,-1.806995e-10,-1.037966e-10,...,-5.514670e-01,-5.270120e-02,-4.579570e-01,-5.203000e-01,-9.093410e-02,-6.145978e-01,1.025021e-10,1.471836e-02,-3.986447e-01,-3.174148e-01
50%,1.254509e+07,-1.944279e-01,-6.985089e-02,-2.355969e-12,-3.057161e-01,-1.326641e-01,-3.143712e-10,-2.518289e-01,-1.806995e-10,-1.037966e-10,...,8.001382e-02,-4.827958e-02,-4.579570e-01,-5.203000e-01,-9.093410e-02,-1.127529e-02,1.025021e-10,1.471836e-02,-1.832882e-01,-1.477679e-01
75%,1.409712e+07,7.352143e-01,7.154424e-01,8.353089e-01,7.158490e-01,1.624382e-01,-3.143712e-10,7.795589e-01,-1.806995e-10,-1.037966e-10,...,7.303425e-01,-2.898243e-02,-4.579570e-01,-5.203000e-01,-7.394897e-12,7.213307e-01,1.025021e-10,1.471836e-02,1.130210e-01,8.632733e-02
max,1.696019e+08,1.653913e+01,1.328013e+01,4.272191e+00,1.808246e+01,5.273837e+02,5.246187e+01,1.831315e+01,6.984489e+01,5.516671e+01,...,1.879033e+00,1.020644e+03,2.183610e+00,3.334287e+01,4.998253e+02,2.186543e+00,1.536751e+02,2.721748e+01,2.045611e+02,3.780532e+02


In [20]:
train_df = pd.read_csv('../data/train_2016_v2.csv')

In [ ]:
train_df

In [21]:
join_df = train_df.merge(house_new_df, how='left', on='parcelid')

In [ ]:
join_df

In [22]:
# training data set: prediction target

target = join_df.loc[:,'logerror']

In [23]:
# training data set: attributes

train = join_df.iloc[:,2:]

# feature = join_df.iloc[:,2:]

In [24]:
# convert datetime into float

for num,str in enumerate(train.transactiondate):
        date_time = datetime.datetime.strptime(str,'%Y-%m-%d')
        str = time.mktime(date_time.timetuple())
        train.set_value(num,'transactiondate',str)
        
train['transactiondate'] = train['transactiondate'].astype(float)

In [27]:
# Models


'''
model = linear_model.LinearRegression()
model = model.fit(feature, target)
'''


model = linear_model.Ridge(alpha=1.0)
model = model.fit(train, target)

'''
model = RandomForestRegressor(max_depth=6, random_state=0)
model = model.fit(feature, target)
'''

'\nmodel = RandomForestRegressor(max_depth=6, random_state=0)\nmodel = model.fit(feature, target)\n'

In [28]:
model

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [29]:
sample = pd.read_csv("../data/sample_submission.csv")

In [ ]:
sample1 = sample.ParcelId

In [ ]:
sample1.columns

In [ ]:
sample1.columns = ['parcelid']

In [ ]:
sample1 = sample1.to_frame()

In [ ]:
sample1.columns

In [ ]:
sample1.columns

In [30]:
sample1 = sample.ParcelId
sample1 = sample1.to_frame()
sample1.columns = ['parcelid']
sample1.describe()
df_test = sample1.merge(house_new_df, on='parcelid', how='left')

In [31]:
del df_test['parcelid']

In [32]:
def date_transform(df_test):
    for num,str in enumerate(df_test.transactiondate):
        date_time = datetime.datetime.strptime(str,'%Y-%m-%d')
        str = time.mktime(date_time.timetuple())
        df_test.set_value(num,'transactiondate',str)
    df_test['transactiondate'] = df_test['transactiondate'].astype(float) 
    
    test_matrix = df_test.as_matrix()
    return test_matrix

In [33]:
result = sample1.copy()
result.columns = ['ParcelId']


df_test1 = df_test.copy()
df_test1.insert(0,'transactiondate','2016-10-15')
df_test1 = date_transform(df_test1)
pred1 = model.predict(df_test1)
pred1 = np.asarray(pred1)
result.insert(1,'201610',pred1)
del df_test1


df_test2 = df_test.copy()
df_test2.insert(0,'transactiondate','2016-11-15')
df_test2 = date_transform(df_test2)
pred2 = model.predict(df_test2)
pred2 = np.asarray(pred2)
result.insert(2,'201611',pred2)
del df_test2


df_test3 = df_test.copy()
df_test3.insert(0,'transactiondate','2016-12-15')
df_test3 = date_transform(df_test3)
pred3 = model.predict(df_test3)
pred3 = np.asarray(pred3)
result.insert(3,'201612',pred3)
del df_test3


df_test4 = df_test.copy()
df_test4.insert(0,'transactiondate','2017-10-15')
df_test4 = date_transform(df_test4)
pred4 = model.predict(df_test4)
pred4 = np.asarray(pred4)
result.insert(4,'201710',pred4)
del df_test4


df_test5 = df_test.copy()
df_test5.insert(0,'transactiondate','2017-11-15')
df_test5 = date_transform(df_test5)
pred5 = model.predict(df_test5)
pred5 = np.asarray(pred5)
result.insert(5,'201711',pred5)
del df_test5


df_test6 = df_test.copy()
df_test6.insert(0,'transactiondate','2017-12-15')
df_test6 = date_transform(df_test6)
pred6 = model.predict(df_test6)
pred6 = np.asarray(pred6)
result.insert(6,'201712',pred6)
del df_test6

# result.columns = ['ParcelId', '201610']

result.columns = ['ParcelId', '201610', '201611', '201612', '201710','201711','201712']
result.to_csv('../predictions/sample5.csv',index=False,header=True)  # need to change filename per run